# MTRec:

Given $I$ number historical clicked news of a user $ N^h = [n_1^h , n_2^h, ..., n^h_I ]$ and a set of $J$ candidate news $ N^c = [n^c_1, n^c_2, ..., n^c_J ] $, our goal is to calculate the user interest score $s_j$ of each candidate news according to the historical behavior of the user, then the candidate news with the highest interest score is recommended to the user. 

For each news, we have its title text T , category label $p^c$, and entity set E. 

## 2.1 News Recommendation Framework

As shown in Figure 2, there are three main components in news recommendation framework, i.e., a news encoder, a user encoder, and a click predictor. 
### News Encoder
For each news n, we encode its title with pre-trained BRET (Devlin et al., 2019). Specifically, we feed the tokenized text T into the BERT model and **adopt the embedding of [CLS] token as the news representation r**. 

We denote the encoded vectors of historical clicked news $N^h$ and candidate news $N^c$ as $R^h = [r^h_1 , r^h_2 , ..., r^h_I ]$ and $R^c = [r^c_1, r^c_2, ..., r^c_J ]$, respectively. 

In [2]:
from torch.utils.data import Dataset, DataLoader
from ebrec.utils._constants import DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL, DEFAULT_INVIEW_ARTICLES_COL, DEFAULT_CLICKED_ARTICLES_COL, DEFAULT_IMPRESSION_ID_COL, DEFAULT_ARTICLE_ID_COL, DEFAULT_TITLE_COL, DEFAULT_BODY_COL
from ebrec.utils._behaviors import truncate_history, create_binary_labels_column
from ebrec.utils._polars import slice_join_dataframes
import polars as pl

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer, RobertaModel, RobertaTokenizer, XLMRobertaModel, XLMRobertaTokenizer

# NOTE: We need the multilingual model for the dataset
model_name = "FacebookAI/xlm-roberta-base"
bert = XLMRobertaModel.from_pretrained(model_name)
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)


COLUMNS = [
    DEFAULT_USER_COL,
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_IMPRESSION_ID_COL,
]


/Users/Matey/project/MTRec-RecSys/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/Matey/project/MTRec-RecSys/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
from math import ceil
from ebrec.utils._constants import (
    DEFAULT_ARTICLE_ID_COL,
    DEFAULT_TITLE_COL,
    DEFAULT_BODY_COL,
    DEFAULT_SUBTITLE_COL,
    DEFAULT_TOPICS_COL,
    DEFAULT_CATEGORY_STR_COL,
    DEFAULT_LABELS_COL
)
DEFAULT_TOKENS_COL = "tokens"
N_SAMPLES_COL = "n_samples"

from ebrec.utils._python import (
    generate_unique_name,
    repeat_by_list_values_from_matrix,
    create_lookup_objects,
    create_lookup_dict,
)

# Honestly, massive respect to that guy for writing all of these utilities but I REALLY REALLY prefer not trying to scour them from 10 different files. Better code quality would be to have it all inside the function. DRY is overrated.

from pprint import pprint

import numpy as np


def map_list_article_id_to_value(
    behaviors: pl.DataFrame,
    behaviors_column: str,
    mapping: dict[int, pl.Series],
    drop_nulls: bool = False,
    fill_nulls: any = None,
) -> pl.DataFrame:
    """

    Maps the values of a column in a DataFrame `behaviors` containing article IDs to their corresponding values
    in a column in another DataFrame `articles`. The mapping is performed using a dictionary constructed from
    the two DataFrames. The resulting DataFrame has the same columns as `behaviors`, but with the article IDs
    replaced by their corresponding values.

    Args:
        behaviors (pl.DataFrame): The DataFrame containing the column to be mapped.
        behaviors_column (str): The name of the column to be mapped in `behaviors`.
        mapping (dict[int, pl.Series]): A dictionary with article IDs as keys and corresponding values as values.
            Note, 'replace' works a lot faster when values are of type pl.Series!
        drop_nulls (bool): If `True`, any rows in the resulting DataFrame with null values will be dropped.
            If `False` and `fill_nulls` is specified, null values in `behaviors_column` will be replaced with `fill_null`.
        fill_nulls (Optional[any]): If specified, any null values in `behaviors_column` will be replaced with this value.

    Returns:
        pl.DataFrame: A new DataFrame with the same columns as `behaviors`, but with the article IDs in
            `behaviors_column` replaced by their corresponding values in `mapping`.

    Example:
    >>> behaviors = pl.DataFrame(
            {"user_id": [1, 2, 3, 4, 5], "article_ids": [["A1", "A2"], ["A2", "A3"], ["A1", "A4"], ["A4", "A4"], None]}
        )
    >>> articles = pl.DataFrame(
            {
                "article_id": ["A1", "A2", "A3"],
                "article_type": ["News", "Sports", "Entertainment"],
            }
        )
    >>> articles_dict = dict(zip(articles["article_id"], articles["article_type"]))
    >>> map_list_article_id_to_value(
            behaviors=behaviors,
            behaviors_column="article_ids",
            mapping=articles_dict,
            fill_nulls="Unknown",
        )
        shape: (4, 2)
        ┌─────────┬─────────────────────────────┐
        │ user_id ┆ article_ids                 │
        │ ---     ┆ ---                         │
        │ i64     ┆ list[str]                   │
        ╞═════════╪═════════════════════════════╡
        │ 1       ┆ ["News", "Sports"]          │
        │ 2       ┆ ["Sports", "Entertainment"] │
        │ 3       ┆ ["News", "Unknown"]         │
        │ 4       ┆ ["Unknown", "Unknown"]      │
        │ 5       ┆ ["Unknown"]                 │
        └─────────┴─────────────────────────────┘
    >>> map_list_article_id_to_value(
            behaviors=behaviors,
            behaviors_column="article_ids",
            mapping=articles_dict,
            drop_nulls=True,
        )
        shape: (4, 2)
        ┌─────────┬─────────────────────────────┐
        │ user_id ┆ article_ids                 │
        │ ---     ┆ ---                         │
        │ i64     ┆ list[str]                   │
        ╞═════════╪═════════════════════════════╡
        │ 1       ┆ ["News", "Sports"]          │
        │ 2       ┆ ["Sports", "Entertainment"] │
        │ 3       ┆ ["News"]                    │
        │ 4       ┆ null                        │
        │ 5       ┆ null                        │
        └─────────┴─────────────────────────────┘
    >>> map_list_article_id_to_value(
            behaviors=behaviors,
            behaviors_column="article_ids",
            mapping=articles_dict,
            drop_nulls=False,
        )
        shape: (4, 2)
        ┌─────────┬─────────────────────────────┐
        │ user_id ┆ article_ids                 │
        │ ---     ┆ ---                         │
        │ i64     ┆ list[str]                   │
        ╞═════════╪═════════════════════════════╡
        │ 1       ┆ ["News", "Sports"]          │
        │ 2       ┆ ["Sports", "Entertainment"] │
        │ 3       ┆ ["News", null]              │
        │ 4       ┆ [null, null]                │
        │ 5       ┆ [null]                      │
        └─────────┴─────────────────────────────┘
    """
    GROUPBY_ID = generate_unique_name(behaviors.columns, "_groupby_id")
    behaviors = behaviors.lazy().with_row_index(GROUPBY_ID)
    # =>
    select_column = (
        behaviors.select(pl.col(GROUPBY_ID), pl.col(behaviors_column))
        .explode(behaviors_column)
        .with_columns(pl.col(behaviors_column).replace(mapping, default=None))
        .collect()
    )
    # =>
    if drop_nulls:
        select_column = select_column.drop_nulls()
    elif fill_nulls is not None:
        select_column = select_column.with_columns(
            pl.col(behaviors_column).fill_null(fill_nulls)
        )
    # =>
    select_column = (
        select_column.lazy().group_by(GROUPBY_ID).agg(behaviors_column).collect()
    )
    return (
        behaviors.drop(behaviors_column)
        .collect()
        .join(select_column, on=GROUPBY_ID, how="left")
        .drop(GROUPBY_ID)
    )


class NewsDataset(Dataset):
    behaviors: pl.DataFrame
    history: pl.DataFrame
    articles: pl.DataFrame

    def __init__(
        self,
        tokenizer,
        behaviors: pl.DataFrame,
        history: pl.DataFrame,
        articles: pl.DataFrame,
        history_size: int = 30,
        padding_value: int = 0,
        max_length=128,
        batch_size=32,
        embeddings_path = None,
    ):
        self.behaviors = behaviors
        self.history = history
        self.articles = articles

        self.history_size = history_size
        self.padding_value = padding_value
        self.batch_size = batch_size

        # self.tokenizer = tokenizer
        # self.max_length = max_length
        
        # TODO: I decided to instead only use pre-computed embeddings for now. You might want to look into this later down the line and implement custom embeddings (and e.g. train BERT as well).
        self.embeddings_path = embeddings_path

        # NOTE: Keep an eye on this if memory issues arise
        self.articles = self.articles.select(
            [
                DEFAULT_ARTICLE_ID_COL,
                DEFAULT_TITLE_COL,
                DEFAULT_BODY_COL,
                DEFAULT_SUBTITLE_COL,
                DEFAULT_TOPICS_COL,
                DEFAULT_CATEGORY_STR_COL,
            ]
        ).collect()

        self._process_history()
        self._set_data()

    def _process_history(self):
        self.history = (
            self.history.select(DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL)
            .pipe(
                truncate_history,
                column=DEFAULT_HISTORY_ARTICLE_ID_COL,
                history_size=self.history_size,
                padding_value=self.padding_value,
                enable_warning=False,
            )
            .collect()
        )
        
    def _set_data(self):
        self.behaviors = self.behaviors.collect()
        self.data: pl.DataFrame = (
            slice_join_dataframes(
                self.behaviors,
                self.history,
                on=DEFAULT_USER_COL,
                how="left",
            )
            .select(COLUMNS)
            .pipe(create_binary_labels_column, seed=42, label_col=DEFAULT_LABELS_COL)
        ).with_columns(
            pl.col(DEFAULT_LABELS_COL).list.len().alias(N_SAMPLES_COL)
        )

        assert self.embeddings_path is not None, "You need to provide a path to the embeddings file."

        embeddings = pl.scan_parquet(self.embeddings_path)

        self.articles = self.articles.lazy().join(embeddings, on=DEFAULT_ARTICLE_ID_COL, how="inner").rename({"FacebookAI/xlm-roberta-base": DEFAULT_TOKENS_COL}).collect()

        article_dict = create_lookup_dict(
            self.articles.select(DEFAULT_ARTICLE_ID_COL, DEFAULT_TOKENS_COL),
            key=DEFAULT_ARTICLE_ID_COL,
            value=DEFAULT_TOKENS_COL,
        )

        self.lookup_indexes, self.lookup_matrix = create_lookup_objects(
            article_dict, unknown_representation="zeros"
        )

    def __len__(self):
        """
        Number of batch steps in the data
        """
        return int(ceil(self.behaviors.shape[0] / self.batch_size))

    def __getitem__(self, index: int):
        """
        Get the batch of samples for the given index.

        Note: The dataset class provides a single index for each iteration. The batching is done internally in this method
        to utilize and optimize for speed. This can be seen as a mini-batching approach.

        Args:
            index (int): An integer index.

        Returns:
            Tuple[torch.Tensor, torch.Tensor]: A tuple containing the input features and labels as torch Tensors.
                Note, the output of the PyTorch DataLoader is (1, *shape), where 1 is the DataLoader's batch_size.
        """
        # Clever way to batch the data:
        batch_indices = range(index * self.batch_size, (index + 1) * self.batch_size)
        batch = self.data[batch_indices]

        x = (
            batch.drop(DEFAULT_LABELS_COL)
            .pipe(
                map_list_article_id_to_value,
                behaviors_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
                mapping=self.lookup_indexes,
                fill_nulls=[0],
            )
            .pipe(
                map_list_article_id_to_value,
                behaviors_column=DEFAULT_INVIEW_ARTICLES_COL,
                mapping=self.lookup_indexes,
                fill_nulls=[0],
            )
        )
        # =>
        repeats = np.array(batch[N_SAMPLES_COL])
        # =>
        history_input = repeat_by_list_values_from_matrix(
            input_array=x[DEFAULT_HISTORY_ARTICLE_ID_COL].to_list(),
            matrix=self.lookup_matrix,
            repeats=repeats,
        ).squeeze(2)
        # =>
        candidate_input = self.lookup_matrix[
            x[DEFAULT_INVIEW_ARTICLES_COL].explode().to_list()
        ]
        # =>
        history_input = torch.tensor(history_input)
        candidate_input = torch.tensor(candidate_input)
        y = torch.tensor(batch[DEFAULT_LABELS_COL].explode(), dtype=torch.float32).view(
            -1, 1
        )
        # ========================
        return history_input, candidate_input, y


import os.path


def load_data(tokenizer, data_path, split="train", embeddings_path=None):
    _data_path = os.path.join(data_path, split)

    df_behaviors = pl.scan_parquet(_data_path + "/behaviors.parquet")
    df_history = pl.scan_parquet(_data_path + "/history.parquet")
    df_articles = pl.scan_parquet(data_path + "/articles.parquet")

    return NewsDataset(tokenizer, df_behaviors, df_history, df_articles, embeddings_path=embeddings_path)


data_path = "../dataset/data/ebnerd_demo"
dataset = load_data(tokenizer, data_path, split="train", embeddings_path="../dataset/data/FacebookAI_xlm_roberta_base/xlm_roberta_base.parquet")

In [4]:
dataset[13][-1]

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
      

In [5]:

class NewsEncoder(nn.Module):
    def __init__(self, bert):
        super(NewsEncoder, self).__init__()
        self.bert = bert

    def forward(self, input_ids):
        outputs = self.bert(input_ids)
        outputs = outputs[0]
        return outputs

### User Encoder
To gain a user representation from the representations of historical clicked news, existing methods usually employ sequential (An et al., 2019) or attentive models (Wu et al., 2019d; Li et al., 2018). In this paper, we adopt additive attention as the user encoder to compress the historical information Rh. The user representation $r^u$ is then denoted as: 

$$ r^u = \sum_{i=1}^I a^u_i r^h_i , a^u_i = \text{softmax}(q^u·\tanh(W^u r^h_i )),$$

 where qu and Wu are trainable parameters. 
 

In [29]:
class MTRec(nn.Module):
    def __init__(self, hidden_dim):
        super(MTRec, self).__init__()

        self.W = nn.Linear(hidden_dim, hidden_dim)
        self.q = nn.Parameter(torch.randn(hidden_dim))

    def forward(self, history, candidates):
        '''
            B - batch size (keep in mind we use an unusual mini-batch approach)
            H - history size (number of articles in the history, usually 30)
            D - hidden size (768)
            history:    B x H x D 
            candidates: B x 1 x D
        '''

        # print(f"{candidates.shape=}")
        att = self.q * F.tanh(self.W(history))
        att_weight = F.softmax(att, dim=1)
        # print(f"{att_weight.shape=}")

        user_embedding = torch.sum(history * att_weight, dim = 1)
        # print(f"{user_embedding.shape=}")
        # print(f"{user_embedding.unsqueeze(-1).shape=}")
        score = torch.bmm(candidates, user_embedding.unsqueeze(-1)) # B x M x 1
        # print(score.shape)
        return score.squeeze(-1)

    def reshape(self, batch_news, bz):
        n_news = len(batch_news) // bz
        reshaped_batch = batch_news.reshape(bz, n_news, -1)
        return reshaped_batch
    
model = MTRec(hidden_dim=768)


In [31]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
dataloader = DataLoader(dataset, batch_size=1)

for i, batch in enumerate(dataloader):
    

    history, candidates, labels = batch
    # print(f"{history.shape=}")
    # print(f"{candidates.shape=}")
    # print(f"{labels.shape=}")
    
    history.squeeze_(0)
    candidates.squeeze_(0)
    labels.squeeze_(0)

    out = model(history, candidates)
    loss = F.cross_entropy(out, labels)
    optimizer.zero_grad(    )
    loss.backward()
    optimizer.step()
    print(f"Step: {i:>3d}:  Loss: {loss.item():.4f}", end='\r')


KeyboardInterrupt: 


### Click Predictor
For each candidate news, we obtain its interest score $s_j$ by matching the candidate news vector $r^c_j$ and the user representation $r^u$ via dot product: $s_j = r^c_j · r^u$. 

### Loss Function
Following previous work (Huang et al., 2013; Wu et al., 2019d), we employ the NCE loss to train the main ranking model. Then the main task loss LM ain is the negative log-likelihood of all positive samples in the training dataset D: 

$$ \mathcal{L}_{Main} = − \sum^{|D|}_{i=1} \log{\exp(s^+_i ) \over \exp(s^+_i ) + \sum^L_{j=1} \exp(s^j_i )} $$ 

where $s^+$ denotes the interest scores of positive news, $L$ indicates the number of negative news.